In [1]:
import os
import numpy as np
import cv2
import torch
import torchvision
import matplotlib.pylab as plt

DATAS_FORWARD_PATH='./pre_test_data/forward/'
DATAS_LEFT_PATH='./pre_test_data/left/'
DATAS_RIGHT_PATH='./pre_test_data/right/'

In [2]:
forward_list=list()
left_list=list()
right_list=list()

In [3]:
dataset=list()
PATH_LIST=[DATAS_FORWARD_PATH,DATAS_LEFT_PATH,DATAS_RIGHT_PATH]
CLASSES_NUM = 3 # go, left, right
label_vector = np.eye(CLASSES_NUM)
for path in PATH_LIST:
    label_name = path.split('/')[-2]
    file_names = os.listdir(path)
    for file_name in file_names:
        if file_name[-3:]=='png':
            print(file_name, label_name)
            img = cv2.imread(path+file_name)
            img_reverse = cv2.flip(img, 1)
            img = img.reshape(3,224,224)
            img_reverse = img_reverse.reshape(3,224,224)
            if label_name=='forward':
                dataset.append((img, label_vector[0]))
                dataset.append((img_reverse, label_vector[0]))
            elif label_name=='left':
                dataset.append((img, label_vector[1]))
                dataset.append((img_reverse, label_vector[2]))
            elif label_name=='right':
                dataset.append((img, label_vector[2]))
                dataset.append((img_reverse, label_vector[1]))


20211202201139.354815.png forward
20211204144800.645320.png forward
20211203190210.966753.png forward
20211204145302.998262.png forward
20211202201619.213176.png forward
20211202201252.661568.png forward
20211202201503.931545.png forward
20211202201137.387277.png forward
20211202201726.894134.png forward
20211202201138.455378.png forward
20211202201254.301947.png forward
20211204145208.699378.png forward
20211202201103.776314.png forward
20211204144709.330430.png forward
20211204145221.083569.png forward
20211204145638.975096.png forward
20211202200955.620214.png forward
20211202201520.817628.png forward
20211204145626.353692.png forward
20211203185342.727912.png forward
20211202200901.652010.png forward
20211202201653.820013.png forward
20211202200956.521482.png forward
20211204144817.702416.png forward
20211202201132.055320.png forward
20211202201002.823979.png forward
20211204145450.532734.png forward
20211202201522.482660.png forward
20211202201742.263154.png forward
20211203185707

In [4]:
dataset[0][1]

array([1., 0., 0.])

In [5]:
TEST_RATE = 0.2
BATCH_SIZE = 8
TEST_SET_NUM = int(len(dataset)*TEST_RATE)
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [len(dataset) - TEST_SET_NUM, TEST_SET_NUM])

In [6]:
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
#     num_workers=4
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
#     num_workers=4
)

In [7]:
class AutoDrive(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.output_classes=3 # position_bool, people, x, y
        self.model_up = self.making_transfer_model1()

    
    def making_transfer_model1(self):
        
        model = torchvision.models.efficientnet_b0(pretrained=True)       
        model.classifier=torch.nn.Sequential(
            torch.nn.Dropout(p=0.5, inplace=True),
            torch.nn.Linear(1280, self.output_classes),
            torch.nn.Softmax(-1),
        )
        return model

    def forward(self, up_img): # follow / to_position / line_num / loaded
        x_up = self.model_up(up_img)
        return x_up


In [8]:
device = 'cuda'
auto = AutoDrive()
auto = auto.to(device=device)

In [9]:
import datetime
from torch.utils.tensorboard import SummaryWriter
LEARNING_RATE = 1e-3 # 1e-3
NUM_EPOCHS=100*1000
optimizer = torch.optim.Adam(auto.parameters(), lr=LEARNING_RATE, betas=(0.0, 0.9), weight_decay=0.0005)
# scaler = torch.cuda.amp.GradScaler()
BEST_MODEL_PATH = 'best_model_effi_down1.pth'
# 3 : lr = 1e-3 early bird 100
# 4 : lr = 1e-3 early bird 1000
criterion = torch.nn.SmoothL1Loss()

d_time = datetime.datetime.now()
folder_name = "runs/"+d_time.strftime("%Y%m%d%H%M%S")
os.mkdir(folder_name)
writer = SummaryWriter(log_dir=folder_name)
pre_test_loss = 10.
EARLY_BIRD = 0
for epoch in range(NUM_EPOCHS):
    train_loss=0.
    test_loss=0.
#     auto.train()
    for up_img,labels in train_loader:
#         print(up_img)
#         up_img = normalize(up_img.float()).to(device=device)
        up_img = up_img.float().to(device=device)
        labels=labels.to(device);                
        optimizer.zero_grad()
        outputs = auto(up_img.float())
#         print("outputs: ",outputs[-1], "labels: ", labels[-1])
#         loss = F.cross_entropy(outputs, labels)
        loss = criterion(outputs, labels)
        train_loss+=loss        
        loss.backward()
        optimizer.step()
#         scaler.scale(loss).backward()
#         scaler.step(optimizer)
#         scaler.update()
        
    
    acc = 0.
    acc0=acc1=acc2=acc3=acc4=0
    count0=count1=count2=count3=count4=0
    with torch.no_grad():
        for up_img,labels in test_loader:
            up_img = up_img.float().to(device=device)
            labels=labels.to(device);        1        
            outputs = auto(up_img.float())    
            loss =criterion(outputs, labels)
            acc0 += sum(labels.argmax(-1)==0)
            acc1 += sum(labels.argmax(-1)==1)
            acc2 += sum(labels.argmax(-1)==2)
            # acc3 += sum(labels.argmax(-1)==3)
            # acc4 += sum(labels.argmax(-1)==4)
            count0 += sum(outputs.argmax(-1)==0)
            count1 += sum(outputs.argmax(-1)==1)
            count2 += sum(outputs.argmax(-1)==2)
            # count3 += sum(outputs.argmax(-1)==3)
            # count4 += sum(outputs.argmax(-1)==4)
                
            acc += sum(outputs.argmax(-1)==labels.argmax(-1))
            test_loss+=loss
            
#             print("outputs: ",outputs)
#     print(f"{epoch} TRAIN_LOSS: {train_loss.item()} TEST LOSS: {test_loss.item()}")
    writer.add_scalar("SUMMARY/ACC",acc.item()/len(test_dataset),epoch)
    writer.add_scalar("SUMMARY/TEST LOSS",test_loss.item()/len(test_dataset),epoch)
    writer.add_scalar("DETAIL/LABEL 0(직진) TEST LOSS",count0.item()/acc0.item(),epoch)
    writer.add_scalar("DETAIL/LABEL 1(좌회전) TEST LOSS",count1.item()/acc1.item(),epoch)
    writer.add_scalar("DETAIL/LABEL 2(우회전) TEST LOSS",count2.item()/acc2.item(),epoch)
    # writer.add_scalar("DETAIL/LABEL 3(사람) TEST LOSS",count3.item()/acc3.item(),epoch)
    # writer.add_scalar("DETAIL/LABEL 4(포지션) TEST LOSS",count4.item()/acc4.item(),epoch)

    # print(epoch, "TRAIN_LOSS",train_loss.item()/len(test_dataset),"TEST LOSS",test_loss.item()/len(test_dataset), 'ACC:', acc.item()/len(test_dataset))
    # print(acc0.item(),count0.item(),"  ", acc1.item(), count1.item(), "  ", acc2.item(), count2.item())
   
#     print("outputs: ",outputs[-1], "labels: ", labels[-1])
    
    if pre_test_loss>test_loss and epoch>50:
        pre_test_loss= test_loss
        torch.save(auto.state_dict(), BEST_MODEL_PATH)
    if pre_test_loss<test_loss and epoch>50:
        EARLY_BIRD +=1
        if EARLY_BIRD >100:
            break